# Australian MV-LV Networks

## 1. Introduction

This repository provides four real large-scale three-phase Australian MV (22kV L-L) distribution networks and corresponding pseudo LV networks (European-style three-phase networks for urban and two-phase networks for rural) down to the connection point of single-phase customers (the pseudo LV networks have been created following modern Australian LV design principles [[1]](https://www.researchgate.net/publication/344346531_On_the_role_of_integrated_MV-LV_network_modelling_in_DER_studies)). These networks are run and operated by [AusNet Services](https://www.ausnetservices.com.au/), a distribution company in the State of Victoria. This repository also includes a large pool of anonymised real 30-min resolution residential demand (kW) profiles. Furthermore, this repository provides the code necessary to extract network data, run time-series power flow simulations, and visualise the results including geographical data, voltage profiles, and asset utilisation.

The table below presents some of the main characteristics of the four Australian distribution networks. The "Name" corresponds to the code/ID used by AusNet Services. "C&I" stands for Commercial and Industrial.

| Network | Type | Name | Nominal Voltages (L-L for 3ph, L-N for SWER) | Total MV Lines | Dist. Trafos | Residential Customers | C&I Customers |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | Rural | _SMR8_ | 22kV, 0.4kV and SWER (12.7kV and 0.23kV) | 680 km | 704 | 3,608 | 61 |
| 2 | Rural | _KLO14_ | 22kV, 0.4kV and SWER (12.7kV and 0.23kV) | 329 km | 700 | 4,691 | 24 |
| 3 | Urban | _HPK11_ | 22kV and 0.4kV | 20 km | 44 | 5,274 | 1 |
| 4 | Urban | _CRE21_ | 22kV and 0.4kV | 30 km | 79 | 3,374 | 9 |

> <span style="color:purple">*Want to learn more about **MV-LV networks and pseudo LV networks**?* Check out this paper [[1]](https://www.researchgate.net/publication/344346531_On_the_role_of_integrated_MV-LV_network_modelling_in_DER_studies) and these reports [[2]](https://www.researchgate.net/publication/334458042_Deliverable_1_HV-LV_modelling_of_selected_HV_feeders), [[3]](https://www.researchgate.net/publication/354907164_Milestone_6_Network_Modelling_and_EV_Impact_Assessment), as well as the webinars of our projects [Advanced Planning of PV-Rich Distribution Networks](https://electrical.eng.unimelb.edu.au/power-energy/projects/pv-rich-distribution-networks) and [EV Integration](https://electrical.eng.unimelb.edu.au/power-energy/projects/ev-integration).</span>

<span style="color:red">**<u>Note</u>.</span>** To make the most of this notebook, you need to understand distribution network analysis and time-series power flows. Ideally, you should have completed all parts of the [Tutorial on DER Hosting Capacity](https://github.com/Team-Nando#1-tutorial-on-der-hosting-capacity).

### Structure of this Notebook
The rest of this notebook is divided into four parts:

- **2. Pre-Run Check.** This part checks if all the necessary files are in place to run the code successfully.
- **3. Network Selection.** Here you will select one of the four networks.
- **4. OpenDSS Circuit.** This part builds the corresponding OpenDSS circuit model.
- **5. Power Flow Analysis.** Finally, this part runs the power flow analysis in two distinct ways: **Snapshot** (a power flow at a specific time, e.g., 12:00pm) and **Time Series** (a daily power flow using specific intervals, e.g., every 15 min). It also includes visualisation of the results such as voltage profiles and asset utilisation.

## 2. Pre-Run Check

### 2.1 Libraries and Functions

First, we import the necessary libraries and the defined functions from the `backend` file in the notebook.

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import geopandas as gp
import dss as dss_direct
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
from colorama import Fore, Style
from tqdm import tqdm
from tabulate import tabulate
from shapely.geometry import Point, LineString
from datetime import datetime, timedelta

#Functions
from backend import *

### 2.2 File Check

Then, we run a script to check that all the necessary files are in place.
- **If the files are detected.** The script adjusts the working directory to include the path for all the files for a smooth execution.
- **If the files are not found.** An error message is displayed in red, indicating that the specified data files are not present in the current directory. You need to address the issue before proceeding with the rest of the code.

<span style="color:red">**<u>Note</u>.</span>** At this stage, you should have copied the two `.npy` files from the `Profiles.zip` to the same folder where the MS Excel files are placed. These profiles correspond to residential and commercial load profiles that will be used later.

In [ ]:
check_data()

## 3. Network Selection

This script allows you to select the network of interest and extract the data from the corresponding MS Excel file. This script uses Python classes and definitions implemented in the `backend` file.

The available networks are listed below. In the next cell, enter the number (1, 2, 3 or 4) of the network to study.

1. `Network_1_Rural_SMR8`

2. `Network_2_Rural_KL014`

3. `Network_3_Urban_HPK11`

4. `Network_4_Urban_CRE21`

In [ ]:
# Select here the network option (1, 2, 3 or 4)
network_option = "1"

### Data Extraction and Visualisation of Network Characteristics

Each sheet in the corresponding MS Excel file contains the necessary data (information and parameters) to properly model different network elements. The name of the sheets and their corresponding information are listed below.
  - **buscoords**: This contains the geographic coordinates of all the MV nodes in the network.
  - **connections**: Details the connections between nodes, including the distances between them.
  - **mvcaps**: Provides data about any capacitors in the network, if they are included.
  - **mvtx**: Contains data about MV/MV transformers and voltage regulators, if present.
  - **lines**: Describes the characteristics of the lines in the MV network.
  - **lvtx**: Offers the data of the distribution transformers (MV/LV) in the circuit.
  - **mv_net_txs**: Has the data of the substation transformer feeding the circuit.
  - **linecodes**: Maps each type of conductor to its standardised parameter values.
  - **lv_lines**: Describes the characteristics of the lines in the LV network.
  - **lv_loads**: Has the data of the loads in the LV circuit.

The output of this code corresponds to two important variables: **network_data** and **network_gis**.
  - **network_data**: A dictionary including the information of all types of elements in different [dataframes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).  
  - **network_gis**: This dictionary also includes the corresponding geographical coordinates of some of the elements of the network in different [geodataframes](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html). 
    
After the data extraction, a table will be displayed. This table outlines some characteristics of the selected network, such as the number of the different elements in the network, and the total lengths of all MV lines and LV lines. Additionally, it also shows a single-line diagram of the MV network, providing a plot to visualise the geographical distribution of the lines and transformers, as well as its topology.

In [ ]:
def identify_network(user_input):
    """
    Prompt the user to enter a network name, convert it to uppercase, and print the identification process.
    """
    
    while True:
        try:
            checkmark = '\u2713'.encode('utf-8').decode('cp1252')
            options = ["1", "2", "3", "4"]
            net_names = ["Network_1", "Network_2", "Network_3", "Network_4"]
            
            if str(user_input).upper() in options:
                print("\033[1;92mUser requested network is: ", net_names[options.index(str(user_input))], "\033[0m")
                current_directory = os.getcwd()
                network_prefix = net_names[options.index(str(user_input))].split("_")
                
                # Search for Excel files matching the network name
                pattern = f"{current_directory}/{network_prefix[0]}_{network_prefix[1]}_*.xlsx"
                
                matched_files = glob.glob(pattern)
                
                if matched_files:
                    network_name = os.path.basename(matched_files[0])
                    data_folder_path = os.path.join(current_directory, network_name)
                    print("\033[1;92mThe circuit's information file is': ", network_name, "\033[0m")
                    all_data = NetworkData(name=str(user_input), data_location=data_folder_path)
                    data = all_data.get_network_data()
                    network_gp = all_data.network_plotting()
                    break
                else:
                    raise MissingExcelSheetsError(f"No Excel files found matching the network: {network_prefix}")
                
            else:
                raise WrongNetworkNameError(
                    f"There is no network associated with the number '{atr(user_input)}' in this repository."
                    f"The available options are 1, 2, 3 or 4. Please re-enter an option."
                )

        except WrongNetworkNameError as e:
            print(f"User Input Error: {e}")
        except MissingExcelSheetsError as e:
            print(f"Network Data Error: {e}")
    
    return data, network_gp

network_data, network_gis = identify_network(network_option)

## 4. OpenDSS Circuit

Before running the code in Python, the first step is to set up the **dss_engine** which will be used for OpenDSS commands.

In [ ]:
dss = dss_direct.DSS  
dss.Start(0)
dss_text = dss.Text
dss_circuit = dss.ActiveCircuit
dss_solution = dss.ActiveCircuit.Solution

### Building the model

The next step is the construction of the electrical model using the data extracted from the selected network's MS Excel file. This involves integrating all component data to set up the basis for subsequent power flow simulations.

The script in this section builds an OpenDSS circuit model using the `DSSDriver` class in `backend` and the identified data above.
Please note that this code does not use __redirect__ function that is typically used in OpenDSS circuit building. All the elements (MV Transformers, Linecodes, MV Lines, LV Transformers, LV Lines, Customer loadings and loadshapes) will be built under this code, by calling the respective dataframe from the dictionary **network_data** which contains the correspondent information.

A key function in this process is `lv_load_mod` which is responsible not only of creating the loads in the model but also for assigning load profiles to the loads. The function takes an input variable, **selected_day**. When set to 0, the function randomly selects a day between 1 and 365 and applies the corresponding daily load profiles to all network loads. If a specific day is provided as the selected_day value, the load profiles for that particular day are applied instead.

In [ ]:
# Day selection: if the value is 0, the subsequent functions will work to assign a random number between 1 to 365:
selected_day = 0

In [ ]:
print("\033[1;92mNetwork building started.\033[0m")
DSSDriver(network_data, network_gis).basic_opendss_actions()
DSSDriver(network_data, network_gis).voltage_source() # Creates the VSource
DSSDriver(network_data, network_gis).mv_net_tx() # Creates the MV transformers
DSSDriver(network_data, network_gis).line_codes() # Creates the line codes for the conductors in the circuit
DSSDriver(network_data, network_gis).connections() # Creates the lines between MV nodes
try:
    DSSDriver(network_data, network_gis).capacitors() # If the circuit has capacitors, they are included
except:
    pass
try:
    DSSDriver(network_data, network_gis).mv_txs() # If the circuit has voltage regulators, they are included
except:
    pass
DSSDriver(network_data, network_gis).lv_tx() # Creates the LV transformers
DSSDriver(network_data, network_gis).lv_nets() #Creates the LV lines
selected_day = DSSDriver(network_data, network_gis).lv_load_mod(selected_day) #Creates the LV loads and profiles.
DSSDriver(network_data, network_gis).print_selected_date(selected_day)
DSSDriver(network_data, network_gis).other_opendss_commands() # Complementary OpenDSS commands
print("\033[1;92mNetwork building completed.\033[0m")

## 5. Power Flow Analysis

### 5.1 Snapshot Mode

In snapshot mode, the simulation focuses on a single specific moment in time for the selected day. The user must specify the desired time as `HH:MM`. Due to the 30-minute resolution of demand data, the time input will be adjusted to the nearest available time slot (e.g., if the user selects 15:43, the adjusted time would be 15:30). 

In [ ]:
# Selection of time for the simulation
selected_time = "15:43"
sim_hh, sim_mm = DSSDriver.sel_time(selected_time)

After specifying the time, it is possible to proceed with the snapshot simulation. The outputs of this simulation are the following dataframes:
- **V_vals_snap**: It contains the L-N voltage values from all the nodes in the circuit in p.u. The base voltage for MV nodes is set at $\frac{22}{\sqrt{3}} \approx 12.70 \textrm{ kV} $, while for LV nodes, it's $\frac{0.4}{\sqrt{3}} \approx 0.23 \textrm{ kV} $.
- **S_lv_txs_snap**: It stores the utilisation percentage of the LV transformers (denoted in the column "Ut_snap"), as well as the characteristics from each transformer (Nominal voltage level, connection type, rated capacity, etc).
- **Idata_snap**: This dataframe stores the utilisation percentage of the MV lines (denoted in the column "Ut_snap"), as well as the characteristics from each line (Rated ampacity, length, etc.)

In [ ]:
temp_dss_ckt = DSSDriver(network_data, network_gis)

def run_snapshot(temp_dss_ckt, sim_hh, sim_mm):
        
        # Include Energymeter at the substation:
        mv_tx_name = temp_dss_ckt.data["mv_net_txs"].loc[0,'Substation_ID']
        try: 
            temp_dss_ckt.dss_text.Command = f'New EnergyMeter.sub_MV element=Transformer.{mv_tx_name} Terminal=1'
        except: # If the element have been already defined
            pass
            
        # Run the simulation
        temp_dss_ckt.dss_text.Command = 'Set Mode=daily number=1 stepsize=30m'
        temp_dss_ckt.dss_text.Command = 'Set time=(' + sim_hh + ',' + sim_mm + ')'
        temp_dss_ckt.dss_solution.Solve()
        
        # Retrieving the voltage values from all the circuit
        temp_dss_ckt.temp_all_V_values = pd.DataFrame(list(temp_dss_ckt.dss_circuit.AllBusVmagPu), index=list(temp_dss_ckt.dss_circuit.AllNodeNames), columns=['Val'])
        temp_dss_ckt.temp_all_V_values['Distance'] = list(temp_dss_ckt.dss_circuit.AllNodeDistances)
        
        # Retrieving the line current measurement for the network plot
        
        for idx in temp_dss_ckt.gis_data["MV_lines"].index:
            
            temp_dss_ckt.dss_circuit.Lines.Name = temp_dss_ckt.gis_data["MV_lines"].loc[idx, "DSSNAME"]
            temp_phases = temp_dss_ckt.dss_circuit.ActiveElement.Properties("phases").Val
            temp_currents = temp_dss_ckt.dss_circuit.ActiveElement.CurrentsMagAng
            
            if temp_phases == "1": # If it is a single-phase line
                av_current = temp_currents[0]
            else: # If it is a three-phase line
                av_current = (temp_currents[0] + temp_currents[2] + temp_currents[4])/3
            
            temp_dss_ckt.gis_data["MV_lines"].loc[idx, "SNAP_val"] = av_current
            
            # Utilisation calculation:
            temp_dss_ckt.gis_data["MV_lines"].loc[idx, "Ut_snap"] = 100*(av_current/temp_dss_ckt.gis_data["MV_lines"].loc[idx, "Ampacity"])
    
        # Retrieving the tx apparent power measurement for the network plot
        
        for idx in temp_dss_ckt.gis_data["MVLV_txs"].index:
            temp_dss_ckt.dss_circuit.Transformers.Name = temp_dss_ckt.gis_data["MVLV_txs"].loc[idx, "DSSNAME"]
            temp_powers = temp_dss_ckt.dss_circuit.ActiveElement.Powers 
            phases = temp_dss_ckt.dss_circuit.ActiveElement.Properties("phases").Val
            
            if phases == "1":
                temp_p_tx = float(temp_powers[0])
                temp_q_tx = float(temp_powers[1])
                    
            # Load P, Q
            else:
                temp_p_tx = float(temp_powers[0]) + float(temp_powers[2]) + float(temp_powers[4])
                temp_q_tx = float(temp_powers[1]) + float(temp_powers[3]) + float(temp_powers[5])
            
            temp_s_tx = np.sqrt(temp_p_tx**2 + temp_q_tx**2)
                
            temp_dss_ckt.gis_data["MVLV_txs"].loc[idx, "SNAP_val"] = temp_s_tx
            
            # Utilisation calculation:
            temp_dss_ckt.gis_data["MVLV_txs"].loc[idx, "Ut_snap"] = 100*(temp_s_tx/temp_dss_ckt.gis_data["MVLV_txs"].loc[idx, "kvas_primary"])

            imp_columns_I = ["DSSNAME", "Length", "Units", "Ampacity", "SNAP_val", "Ut_snap"]
            imp_columns_Tx = ["Substation_ID", "Connection_Primary", "Connection_Secondary", "kvas_primary", "Type", "Customers", "DSSNAME", "SNAP_val", "Ut_snap"]
        return temp_dss_ckt.temp_all_V_values, temp_dss_ckt.gis_data["MVLV_txs"], temp_dss_ckt.gis_data["MV_lines"]
    
    
V_vals_snap, S_lv_txs_snap, Idata_snap = run_snapshot(temp_dss_ckt, sim_hh, sim_mm)

### 5.2 Snapshot Mode Results

The simulation results can be visualised using the following plots.

- **Voltage Profile Plot**: Depicts node voltages (in p.u.) along the feeder, distinguished by line width for MV and LV nodes (wider points correspond to MV nodes and narrower to LV nodes) and colour for phase connections (Phase A: Red, Phase B: Black, Phase C: Blue). This plot also helps to visualise the voltage drop relative to the distance from the MV substation.

In [ ]:
temp_dss_ckt.Voltge_profile_plot()

- **Transformer Utilisation Map**: A heatmap indicating the utilisation percentage (100 * kVA consumed/nominal kVA) for each LV substation at the chosen time, with a color gradient from green (low utilisation - 0%) to red (high utilisation - 100%).

In [ ]:
temp_dss_ckt.gis_tx_utilisation()


- **Line Utilisation Map**: Similar to the transformer map, this displays the MV line utilisation levels at the chosen time.

In [ ]:
temp_dss_ckt.gis_lines_utilisation()

### 5.3 Time-Series Mode

Time-series mode conducts a full day's simulation across all 48 half-hourly intervals. In this simulation, the voltages from all customers, as well as the utilisation percentage of all the conductors and transformers are being monitored in all the time instances. The outputs are the following dataframes:
- **V_loads**: The voltage values from all the customers in Volts (V).
- **P_loads**: The active power net demand of each customer in kW.
- **Q_loads**: The reactive power net demand of each customer in kvar.
- **S_mv_tx**: The utilisation percentage of the MV substation transformer.
- **S_lv_txs**: The utilisation percentage of each LV transformer.
- **Idata**: The utilisation percentage of the MV lines.

In [ ]:
temp_dss_ckt2 = DSSDriver(data=network_data, gis_data=network_gis)

def run_daily(temp_dss_ckt2):
        
        # Setting the properties for the simulation
        temp_dss_ckt2.dss_text.Command = 'Set Mode=daily number=1 stepsize=30m'
        temp_dss_ckt2.dss_text.Command = 'Set time=(0,0)'
        
        # Get the information from the loads
        loads_name = temp_dss_ckt2.dss_circuit.Loads.AllNames
        buses_name_loads = []
        
        for load in loads_name:

            temp_dss_ckt2.dss_circuit.Loads.Name = load
            temp_load_bus = temp_dss_ckt2.dss_circuit.ActiveElement.Properties("bus1").Val
            n_phases = temp_dss_ckt2.dss_circuit.ActiveElement.Properties("phases").Val
            if n_phases == "1":
                buses_name_loads.append(temp_load_bus.split(".0")[0])
            else:
                for x in [".1", ".2", ".3"]:
                    buses_name_loads.append(temp_load_bus.split(".")[0]+x)
        
        # Get the information from the MV/LV transformers
        txs_name = [x for x in temp_dss_ckt2.dss_circuit.Transformers.AllNames if "mv_f0_lv_" in x]
        
        
        # Output variables
        # Voltage info
        temp_dss_ckt2.Vdata = pd.DataFrame(index=range(48), columns=buses_name_loads)
        # Active Power info
        temp_dss_ckt2.Pdata = pd.DataFrame(index=range(48), columns=loads_name)
        # Reactive Power Info
        temp_dss_ckt2.Qdata = pd.DataFrame(index=range(48), columns=loads_name)
        # Aparent power seen from the main MV tx:
        temp_dss_ckt2.Sdata = pd.DataFrame(index=range(48), columns=[])
        # Aparent power from all the transformers:
        temp_dss_ckt2.Sdata_txs = {} 
        temp_dss_ckt2.Sdata_txs = pd.DataFrame(index=range(48), columns=[])
        # Line currents
        temp_dss_ckt2.Idata = pd.DataFrame(index=range(48), columns=[temp_dss_ckt2.gis_data["MV_lines"].loc[x, "DSSNAME"] for x in temp_dss_ckt2.gis_data["MV_lines"].index])
        
        
        # Initialise an empty list to store Series
        sdata_txs_list = []

        for it in tqdm(range(48), desc="Solving time-series power flow"):
            temp_dss_ckt2.dss_solution.Solve()
            # Getting the results and appending them on the variables V, P and Q data
            
            # Aparent power seen from the MV substation
            temp_dss_ckt2.dss_circuit.Lines.Name = "mv_f0_l0"
            temp_powers = temp_dss_ckt2.dss_circuit.ActiveElement.Powers
            
            temp_p_load = float(temp_powers[0]) + float(temp_powers[2]) + float(temp_powers[4])
            temp_q_load = float(temp_powers[1]) + float(temp_powers[3]) + float(temp_powers[5])
            
            temp_dss_ckt2.Sdata.loc[it, "Value"] = np.sqrt(temp_p_load**2 + temp_p_load**2)
            
            
            # Voltages:
            temp_all_V_values = pd.DataFrame(list(temp_dss_ckt2.dss_circuit.AllBusVmag), index=list(temp_dss_ckt2.dss_circuit.AllNodeNames), columns=['Val'])

            # for all loads
            temp_dss_ckt2.Vdata.loc[it, :] =  temp_all_V_values.loc[buses_name_loads, :].values.flatten()
            
            # Active Power 
            for load in temp_dss_ckt2.dss_circuit.Loads.AllNames:
                    
                temp_dss_ckt2.dss_circuit.Loads.Name = load
                temp_powers = temp_dss_ckt2.dss_circuit.ActiveElement.Powers
                phases = temp_dss_ckt2.dss_circuit.ActiveElement.Properties("phases").Val
                
                # Load P, Q
                if phases == "1":
                    temp_p_load = float(temp_powers[0])
                    temp_q_load = float(temp_powers[1])
                                
                    # PVs P, Q
                    try:
                        temp_dss_ckt2.dss_circuit.PVSystems.Name = load
                        temp_powers = temp_dss_ckt2.dss_circuit.ActiveElement.Powers
                        
                        temp_p_pv = float(temp_powers[0])
                        temp_q_pv = float(temp_powers[1])
                        
                    except:
                        temp_p_pv = 0
                        temp_q_pv = 0
                        
                # Load P, Q
                else:
                    temp_p_load = float(temp_powers[0]) + float(temp_powers[2]) + float(temp_powers[4])
                    temp_q_load = float(temp_powers[1]) + float(temp_powers[3]) + float(temp_powers[5])
                                
                    # PVs P, Q
                    try:
                        temp_dss_ckt2.dss_circuit.PVSystems.Name = load
                        temp_powers = temp_dss_ckt2.dss_circuit.ActiveElement.Powers
                        
                        temp_p_pv = float(temp_powers[0])
                        temp_q_pv = float(temp_powers[1])
                        
                    except:
                        temp_p_pv = 0
                        temp_q_pv = 0
                    
                temp_dss_ckt2.Pdata.loc[it, load] = temp_p_load + temp_p_pv
                temp_dss_ckt2.Qdata.loc[it, load] = temp_q_load + temp_q_pv

            # Analysis for transformers
            
            temp_s_tx_dict = {}
            for tx in txs_name:

                temp_dss_ckt2.dss_circuit.Transformers.Name = tx
                temp_powers = temp_dss_ckt2.dss_circuit.ActiveElement.Powers
                phases = temp_dss_ckt2.dss_circuit.ActiveElement.Properties("phases").Val
                
                if phases == "1":
                    temp_p_tx = float(temp_powers[0])
                    temp_q_tx = float(temp_powers[1])
                        
                # Load P, Q
                else:
                    temp_p_tx = float(temp_powers[0]) + float(temp_powers[2]) + float(temp_powers[4])
                    temp_q_tx = float(temp_powers[1]) + float(temp_powers[3]) + float(temp_powers[5])
                
                temp_s_tx = np.sqrt(temp_p_tx**2 + temp_q_tx**2)
                    
                # Store the apparent power in the dictionary with the transformer name as the key
                temp_s_tx_dict[tx] = temp_s_tx
            
            sdata_txs_list.append(pd.Series(temp_s_tx_dict, name=it))

            # Analysis for lines
            
            for idx in temp_dss_ckt2.gis_data["MV_lines"].index:
                line_name = temp_dss_ckt2.gis_data["MV_lines"].loc[idx, "DSSNAME"].lower()
                temp_dss_ckt2.dss_circuit.Lines.Name = line_name
                temp_phases = temp_dss_ckt2.dss_circuit.ActiveElement.Properties("phases").Val
                temp_currents = temp_dss_ckt2.dss_circuit.ActiveElement.CurrentsMagAng
                
                if temp_phases == "1":
                    av_current = float(temp_currents[0])
                else:
                    av_current = (float(temp_currents[0]) + float(temp_currents[2]) + float(temp_currents[4]))/3
                
                temp_dss_ckt2.Idata.loc[it, line_name] = av_current
        
        # Once the loop is done, concatenate all Series along the columns
        temp_dss_ckt2.Sdata_txs = pd.concat(sdata_txs_list, axis=1)
        # Transpose the DataFrame to have transformers as columns and iterations as rows
        temp_dss_ckt2.Sdata_txs = temp_dss_ckt2.Sdata_txs.T

        # Retrieving the tx apparent power measurement for the network plot
        for idx in temp_dss_ckt2.gis_data["MVLV_txs"].index:
            tx = temp_dss_ckt2.gis_data["MVLV_txs"].loc[idx, "DSSNAME"].lower()
            temp_dss_ckt2.gis_data["MVLV_txs"].loc[idx, "DAILY_max"] = temp_dss_ckt2.Sdata_txs.loc[:,tx].max()
        
        # Retrieving the max current for the network plot
        for idx in temp_dss_ckt2.gis_data["MV_lines"].index:
            line = temp_dss_ckt2.gis_data["MV_lines"].loc[idx, "DSSNAME"].lower()
            temp_dss_ckt2.gis_data["MV_lines"].loc[idx, "DAILY_max"] = temp_dss_ckt2.Idata.loc[:,line].max()

        return temp_dss_ckt2.Vdata, temp_dss_ckt2.Pdata, temp_dss_ckt2.Qdata, temp_dss_ckt2.Sdata, temp_dss_ckt2.Sdata_txs, temp_dss_ckt2.Idata
    
V_loads, P_loads, Q_loads, S_mv_tx, S_lv_txs, Idata = run_daily(temp_dss_ckt2)

### 5.4 Time-Series Mode Results

The simulation results can be visualised using the following plots.

- **Daily Voltage Plot**: Shows the voltage trends (in Volts) for each customer throughout the day, with Australia's statutory limit (253 V, 1.1 p.u.) marked by a red dashed line. A desirable result here is to have all the voltage levels below the maximum statutory limit. 

In [ ]:
temp_dss_ckt2.Voltge_timeseries_plot()

- **Daily Transformer Utilisation Plot**: Presents transformer utilisation over time, highlighting commercial (black) and residential (light grey) substations, and the MV substation (blue). A red dashed line is used to mark the 100% percent of utilisation level. Again, a desirable result is to have all the utilisation levels below the red dashed line. Any transformer utilisation exceeding this value would indicate a state of overload.

In [ ]:
temp_dss_ckt2.Tx_utilisation_plot()

- **Maximum Transformer Utilisation Map**: A heatmap showcasing the peak daily utilisation for each LV substation, with a green-to-red color gradient indicating low to high usage.

In [ ]:
temp_dss_ckt2.gis_tx_utilisation_daily()

- **Maximum Line Utilisation Map**: Similar to the transformer map, illustrating the day's peak MV line utilisation levels.

In [ ]:
temp_dss_ckt2.gis_lines_utilisation_daily()